## Question 2_3
Implemeting tokenizer by BPE and WordPiece

In [ ]:
#wordpiece
import re
from collections import Counter, defaultdict

class WordPieceTokenizer:
    def __init__(self, vocab, max_word_length=20):
        self.vocab = vocab
        self.max_word_length = max_word_length

    def _build_subwords(self):
        subwords = set()
        for word in self.vocab:
            for i in range(len(word)):
                for j in range(i + 1, min(len(word), i + self.max_word_length) + 1):
                    subwords.add(word[i:j])
        return subwords

    def _train_wordpiece(self, num_iterations=10):
        for _ in range(num_iterations):
            subwords = self._build_subwords()
            for subword in subwords:
                if subword not in self.vocab:
                    self.vocab[subword] = len(self.vocab)

    def tokenize(self, text):
        tokens = []
        start = 0
        while start < len(text):
            match = False
            for end in range(min(len(text), start + self.max_word_length), start, -1):
                substr = text[start:end]
                if substr in self.vocab:
                    tokens.append(substr)
                    start = end
                    match = True
                    break
            if not match:
                tokens.append(text[start])
                start += 1
        return tokens

def preprocess_text(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)
    words = text.split()
    return words


file_path = '/content/All_Around_the_Moon.txt'

words = preprocess_text(file_path)
vocab = {word: i for i, word in enumerate(Counter(words))}
tokenizer = WordPieceTokenizer(vocab)
tokenizer._train_wordpiece()
all_tokens = tokenizer.tokenize(' '.join(words))
print(all_tokens[:100])


['the', ' ', 'project', ' ', 'gutenberg', ' ', 'ebook', ' ', 'of', ' ', 'all', ' ', 'around', ' ', 'the', ' ', 'moon', ' ', 'this', ' ', 'ebook', ' ', 'is', ' ', 'for', ' ', 'the', ' ', 'use', ' ', 'of', ' ', 'anyone', ' ', 'anywhere', ' ', 'in', ' ', 'the', ' ', 'united', ' ', 'states', ' ', 'and', ' ', 'most', ' ', 'other', ' ', 'parts', ' ', 'of', ' ', 'the', ' ', 'world', ' ', 'at', ' ', 'no', ' ', 'cost', ' ', 'and', ' ', 'with', ' ', 'almost', ' ', 'no', ' ', 'restrictions', ' ', 'whatsoever', ' ', 'you', ' ', 'may', ' ', 'copy', ' ', 'it', ' ', 'give', ' ', 'it', ' ', 'away', ' ', 'or', ' ', 'reuse', ' ', 'it', ' ', 'under', ' ', 'the', ' ']


In [ ]:
#BPE
import re
from collections import Counter

class BPE:
    def __init__(self, corpus, vocab_size):
        self.vocab_size = vocab_size
        self.vocab = self.train(corpus)

    def train(self, corpus):
        vocab = {(' '.join(word) + ' </w>',): freq for word, freq in corpus.items()}

        while len(self.flatten_vocab(vocab)) < self.vocab_size:
            pairs = self.get_stats(vocab)
            if not pairs:
                break
            best_pair = max(pairs, key=pairs.get)
            vocab = self.merge_best_pair(best_pair, vocab)
        return self.flatten_vocab(vocab)

    def flatten_vocab(self, vocab):
        return set([subword for word in vocab for subword in word])

    def get_stats(self, vocab):
        pairs = Counter()
        for word, freq in vocab.items():
            symbols = word[0].split()
            for i in range(len(symbols)-1):
                pairs[symbols[i], symbols[i+1]] += freq
        return pairs

    def merge_best_pair(self, best_pair, vocab):
        new_vocab = {}
        bigram = ' '.join(best_pair)
        replacer = ''.join(best_pair)
        for word in vocab:
            word_str = word[0]
            new_word = word_str.replace(bigram, replacer)
            new_vocab[(new_word,)] = vocab[word]
        return new_vocab

    def tokenize(self, text):
        text = text.lower()
        text = re.sub(r'[^a-z\s]', '', text)
        words = text.split()
        tokens = []
        for word in words:
            word = ' '.join(word) + ' </w>'
            word_tokens = []
            while word:
                possible_tokens = [token for token in self.vocab if word.startswith(token.replace('</w>', ' </w>'))]
                if not possible_tokens:
                    word_tokens.append(word)
                    break
                longest_token = max(possible_tokens, key=len)
                word_tokens.append(longest_token)
                word = word[len(longest_token):].lstrip()
            tokens.extend(word_tokens)
        return tokens

file_path = '/content/All_Around_the_Moon.txt'
with open(file_path, 'r', encoding='utf-8') as file:
    text_data = file.read()

processed_text = re.sub(r'[^a-z\s]', '', text_data.lower())
words = processed_text.split()
word_freq = Counter(words)
bpe_tokenizer = BPE(word_freq, vocab_size=50000)
tokenized_text = bpe_tokenizer.tokenize(processed_text)
print(tokenized_text[:100])

['t h e </w>', 'p r o j e c t </w>', 'g u t e n b e r g </w>', 'e b o o k </w>', 'o f </w>', 'a l l </w>', 'a r o u n d </w>', 't h e </w>', 'm o o n </w>', 't h i s </w>', 'e b o o k </w>', 'i s </w>', 'f o r </w>', 't h e </w>', 'u s e </w>', 'o f </w>', 'a n y o n e </w>', 'a n y w h e r e </w>', 'i n </w>', 't h e </w>', 'u n i t e d </w>', 's t a t e s </w>', 'a n d </w>', 'm o s t </w>', 'o t h e r </w>', 'p a r t s </w>', 'o f </w>', 't h e </w>', 'w o r l d </w>', 'a t </w>', 'n o </w>', 'c o s t </w>', 'a n d </w>', 'w i t h </w>', 'a l m o s t </w>', 'n o </w>', 'r e s t r i c t i o n s </w>', 'w h a t s o e v e r </w>', 'y o u </w>', 'm a y </w>', 'c o p y </w>', 'i t </w>', 'g i v e </w>', 'i t </w>', 'a w a y </w>', 'o r </w>', 'r e u s e </w>', 'i t </w>', 'u n d e r </w>', 't h e </w>', 't e r m s </w>', 'o f </w>', 't h e </w>', 'p r o j e c t </w>', 'g u t e n b e r g </w>', 'l i c e n s e </w>', 'i n c l u d e d </w>', 'w i t h </w>', 't h i s </w>', 'e b o o k </w>',

Tokeninzing the following sentences:

- This darkness is absolutely killing! If we ever take this trip again, it must be about the time of the sNew Moon!

- This is a tokenization task. Tokenization is the first step in a NLP pipeline. We will be comparing the tokens generated by each tokenization model.

In [ ]:
#Wordpiece
import re
from collections import Counter, defaultdict

class WordPieceTokenizer:
    def __init__(self, vocab, max_word_length=20):
        self.vocab = vocab
        self.max_word_length = max_word_length

    def _build_subwords(self):
        subwords = set()
        for word in self.vocab:
            for i in range(len(word)):
                for j in range(i + 1, min(len(word), i + self.max_word_length) + 1):
                    subwords.add(word[i:j])
        return subwords

    def _train_wordpiece(self, num_iterations=10):
        for _ in range(num_iterations):
            subwords = self._build_subwords()
            for subword in subwords:
                if subword not in self.vocab:
                    self.vocab[subword] = len(self.vocab)

    def tokenize(self, text):
        tokens = []
        start = 0
        while start < len(text):
            match = False
            for end in range(min(len(text), start + self.max_word_length), start, -1):
                substr = text[start:end]
                if substr in self.vocab:
                    tokens.append(substr)
                    start = end
                    match = True
                    break
            if not match:
                tokens.append(text[start])
                start += 1
        return tokens

def preprocess_sentences(sentences):
    processed_sentences = []
    for sentence in sentences:
        sentence = sentence.lower()
        sentence = re.sub(r'[^a-z0-9\s]', '', sentence)
        processed_sentences.extend(sentence.split())
    return processed_sentences


sentences = [
    "This darkness is absolutely killing! If we ever take this trip again, it must be about the time of the sNew Moon!",
    "This is a tokenization task. Tokenization is the first step in a NLP pipeline. We will be comparing the tokens generated by each tokenization model."
]

words = preprocess_sentences(sentences)
vocab = {word: i for i, word in enumerate(Counter(words))}

tokenizer = WordPieceTokenizer(vocab)
tokenizer._train_wordpiece()

all_tokens = []
for sentence in sentences:
    tokens = tokenizer.tokenize(sentence)
    all_tokens.extend(tokens)
print(all_tokens)



['T', 'his', ' ', 'darkness', ' ', 'is', ' ', 'absolutely', ' ', 'killing', '!', ' ', 'I', 'f', ' ', 'we', ' ', 'ever', ' ', 'take', ' ', 'this', ' ', 'trip', ' ', 'again', ',', ' ', 'it', ' ', 'must', ' ', 'be', ' ', 'about', ' ', 'the', ' ', 'time', ' ', 'of', ' ', 'the', ' ', 's', 'N', 'ew', ' ', 'M', 'oon', '!', 'T', 'his', ' ', 'is', ' ', 'a', ' ', 'tokenization', ' ', 'task', '.', ' ', 'T', 'okenization', ' ', 'is', ' ', 'the', ' ', 'first', ' ', 'step', ' ', 'in', ' ', 'a', ' ', 'N', 'L', 'P', ' ', 'pipeline', '.', ' ', 'W', 'e', ' ', 'will', ' ', 'be', ' ', 'comparing', ' ', 'the', ' ', 'tokens', ' ', 'generated', ' ', 'by', ' ', 'each', ' ', 'tokenization', ' ', 'model', '.']


In [ ]:
#BPE
import re
from collections import Counter

class BPE:
    def __init__(self, corpus, vocab_size):
        self.vocab_size = vocab_size
        self.vocab = self.train(corpus)

    def train(self, corpus):
        vocab = {(' '.join(word) + ' </w>',): freq for word, freq in corpus.items()}
        while len(self.flatten_vocab(vocab)) < self.vocab_size:
            pairs = self.get_stats(vocab)
            if not pairs:
                break
            best_pair = max(pairs, key=pairs.get)
            vocab = self.merge_best_pair(best_pair, vocab)
        return self.flatten_vocab(vocab)

    def flatten_vocab(self, vocab):
        return set([subword for word in vocab for subword in word])

    def get_stats(self, vocab):
        pairs = Counter()
        for word, freq in vocab.items():
            symbols = word[0].split()
            for i in range(len(symbols)-1):
                pairs[symbols[i], symbols[i+1]] += freq
        return pairs

    def merge_best_pair(self, best_pair, vocab):
        new_vocab = {}
        bigram = ' '.join(best_pair)
        replacer = ''.join(best_pair)
        for word in vocab:
            word_str = word[0]
            new_word = word_str.replace(bigram, replacer)
            new_vocab[(new_word,)] = vocab[word]
        return new_vocab

    def tokenize(self, text):
        text = text.lower()
        text = re.sub(r'[^a-z\s]', '', text)
        words = text.split()
        tokens = []
        for word in words:
            word = ' '.join(word) + ' </w>'
            word_tokens = []
            while word:
                possible_tokens = [token for token in self.vocab if word.startswith(token.replace('</w>', ' </w>'))]
                if not possible_tokens:
                    word_tokens.append(word)
                    break
                longest_token = max(possible_tokens, key=len)
                word_tokens.append(longest_token)
                word = word[len(longest_token):].lstrip()
            tokens.extend(word_tokens)
        return tokens

sentences = [
    "This darkness is absolutely killing! If we ever take this trip again, it must be about the time of the sNew Moon!",
    "This is a tokenization task. Tokenization is the first step in a NLP pipeline. We will be comparing the tokens generated by each tokenization model."
]

processed_text = ' '.join(sentences).lower()
processed_text = re.sub(r'[^a-z\s]', '', processed_text)

words = processed_text.split()
word_freq = Counter(words)
bpe_tokenizer = BPE(word_freq, vocab_size=500)
tokenized_sentences = []

for sentence in sentences:
    tokenized_sentence = bpe_tokenizer.tokenize(sentence)
    tokenized_sentences.extend(tokenized_sentence)
print(tokenized_sentences)

['t h i s </w>', 'd a r k n e s s </w>', 'i s </w>', 'a b s o l u t e l y </w>', 'k i l l i n g </w>', 'i f </w>', 'w e </w>', 'e v e r </w>', 't a k e </w>', 't h i s </w>', 't r i p </w>', 'a g a i n </w>', 'i t </w>', 'm u s t </w>', 'b e </w>', 'a b o u t </w>', 't h e </w>', 't i m e </w>', 'o f </w>', 't h e </w>', 's n e w </w>', 'm o o n </w>', 't h i s </w>', 'i s </w>', 'a</w>', '>', 't o k e n i z a t i o n </w>', 't a s k </w>', 't o k e n i z a t i o n </w>', 'i s </w>', 't h e </w>', 'f i r s t </w>', 's t e p </w>', 'i n </w>', 'a</w>', '>', 'n l p </w>', 'p i p e l i n e </w>', 'w e </w>', 'w i l l </w>', 'b e </w>', 'c o m p a r i n g </w>', 't h e </w>', 't o k e n s </w>', 'g e n e r a t e d </w>', 'b y </w>', 'e a c h </w>', 't o k e n i z a t i o n </w>', 'm o d e l </w>']


## Question 3_1


In [3]:
import re
import string

with open('/content/Tarzan.txt', 'r', encoding='utf-8') as file:
    text_data = file.read()

text_data = re.sub(r'[^\x00-\x7F]+', ' ', text_data)
text_data = text_data.lower()
translator = str.maketrans('', '', string.punctuation)
text_data = text_data.translate(translator)

tokens = text_data.split()

from collections import Counter
bigrams = zip(tokens[:-1], tokens[1:])

bigram_counts = Counter(bigrams)
bigram_counts.most_common(20)


[(('of', 'the'), 966),
 (('in', 'the'), 372),
 (('to', 'the'), 312),
 (('ibn', 'jad'), 227),
 (('and', 'the'), 190),
 (('upon', 'the'), 162),
 (('from', 'the'), 156),
 (('he', 'had'), 151),
 (('of', 'his'), 134),
 (('of', 'a'), 124),
 (('that', 'he'), 124),
 (('he', 'was'), 119),
 (('it', 'was'), 118),
 (('at', 'the'), 117),
 (('into', 'the'), 115),
 (('with', 'the'), 113),
 (('of', 'nimmr'), 109),
 (('the', 'great'), 99),
 (('by', 'the'), 98),
 (('for', 'the'), 96)]

## Question 3_2

In [ ]:
from collections import Counter, defaultdict
import re
import string

def load_and_preprocess_text(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)
    text = text.lower()
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)
    tokens = text.split()
    return tokens

def count_bigrams_and_tokens(tokens):
    bigrams = zip(tokens[:-1], tokens[1:])
    bigram_counts = Counter(bigrams)
    token_counts = Counter(tokens)
    return bigram_counts, token_counts

def calculate_smoothed_probabilities(bigram_counts, token_counts):
    vocabulary_size = len(set(token_counts))
    smoothed_probabilities = {}
    for bigram, count in bigram_counts.items():
        first_word_count = token_counts[bigram[0]]
        smoothed_count = count + 1
        smoothed_probability = smoothed_count / (first_word_count + vocabulary_size)
        smoothed_probabilities[bigram] = smoothed_probability
    return smoothed_probabilities

def main(file_path):
    tokens = load_and_preprocess_text(file_path)
    bigram_counts, token_counts = count_bigrams_and_tokens(tokens)
    smoothed_probabilities = calculate_smoothed_probabilities(bigram_counts, token_counts)
    print(list(smoothed_probabilities.items())[:10])

file_path = '/content/Tarzan.txt'
main(file_path)


[(('the', 'project'), 0.0027414933075310434), (('project', 'gutenberg'), 0.012837155632482332), (('gutenberg', 'ebook'), 0.0005770340450086555), (('ebook', 'of'), 0.00029167274318214965), (('of', 'tarzan'), 0.002870508186264087), (('tarzan', 'lord'), 0.0015328874024526198), (('lord', 'of'), 0.0018895348837209302), (('of', 'the'), 0.10280671911545822), (('the', 'jungle'), 0.0077406869859700045), (('jungle', 'this'), 0.00028764562059542645)]


## Question 3_3

In [ ]:
from collections import Counter
import re

def preprocess_text(filepath):
    with open(filepath, 'r', encoding='utf-8') as file:
        text = file.read().lower()
        text = re.sub('[^a-z\s]', '', text)
    return text.split()

def count_bigrams(words):
    bigrams = [(words[i], words[i + 1]) for i in range(len(words) - 1)]
    return Counter(bigrams)

def smoothed_probabilities(bigrams, vocabulary_size):
    total_bigrams = sum(bigrams.values()) + vocabulary_size**2
    probabilities = {bigram: (count + 1) / total_bigrams for bigram, count in bigrams.items()}
    return probabilities

def generate_text(start, probabilities, num_words=10):
    current_word = start.split()[-1]
    text = start
    for _ in range(num_words):
        next_words = [bigram[1] for bigram in probabilities if bigram[0] == current_word]
        if not next_words:
            break
        current_word = next_words[0]
        text += ' ' + current_word
    return text

if __name__ == "__main__":
    filepath = '/content/Tarzan.txt'
    words = preprocess_text(filepath)
    bigrams = count_bigrams(words)
    vocabulary_size = len(set(words))
    probabilities = smoothed_probabilities(bigrams, vocabulary_size)

    starting_phrases = [
        "knowing well the windings of the trail he",
        "for half a day he lolled on the huge back and"
    ]

    for phrase in starting_phrases:
        print(generate_text(phrase, probabilities))


knowing well the windings of the trail he threw his great tourney the project gutenberg ebook of tarzan
for half a day he lolled on the huge back and most other parts of tarzan lord of tarzan lord of


## Question 3_4
Rewriting question 3_3, considering n = 3 and n = 5

In [ ]:
from collections import Counter
import re

def preprocess_text(filepath):
    with open(filepath, 'r', encoding='utf-8') as file:
        text = file.read().lower()
        text = re.sub('[^a-z\s]', '', text)
    return text.split()

def count_ngrams(words, n):
    ngrams = [tuple(words[i:i+n]) for i in range(len(words) - n + 1)]
    return Counter(ngrams)

def smoothed_probabilities(ngrams, vocabulary_size):
    total_ngrams = sum(ngrams.values()) + vocabulary_size ** n
    probabilities = {ngram: (count + 1) / total_ngrams for ngram, count in ngrams.items()}
    return probabilities

def generate_text(start, probabilities, n, num_words=10):
    current_words = tuple(start.split()[-(n-1):])
    text = start
    for _ in range(num_words):
        next_words = [ngram[-1] for ngram in probabilities if ngram[:-1] == current_words]
        if not next_words:
            break
        current_words = current_words[1:] + (next_words[0],)
        text += ' ' + current_words[-1]
    return text

if __name__ == "__main__":
    filepath = '/content/Tarzan.txt'
    words = preprocess_text(filepath)

    n_values = [3, 5]

    for n in n_values:
        ngrams = count_ngrams(words, n)
        vocabulary_size = len(set(words))
        probabilities = smoothed_probabilities(ngrams, vocabulary_size)

       starting_phrases = [
            "knowing well the windings of the trail he",
            "for half a day he lolled on the huge back and"
        ]

        print(f"Using {n}-gram:")
        for phrase in starting_phrases:
            print(generate_text(phrase, probabilities, n))


Using 3-gram:
knowing well the windings of the trail he did not hate mennot even white men with powerful express
for half a day he lolled on the huge back and usha tore through the foliage swayed the giant bulk of
Using 5-gram:
knowing well the windings of the trail he took short cuts swinging through the branches of the trees
for half a day he lolled on the huge back and


## Question 3_4
Rewriting question 3_2, considering n = 3 and n = 5

In [ ]:
from collections import Counter, defaultdict
import re
import string

def load_and_preprocess_text(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)
    text = text.lower()
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)
    tokens = text.split()
    return tokens

def count_ngrams_and_tokens(tokens, n):
    ngrams = [tuple(tokens[i:i+n]) for i in range(len(tokens) - n + 1)]
    ngram_counts = Counter(ngrams)
    token_counts = Counter(tokens)
    return ngram_counts, token_counts

def calculate_smoothed_probabilities(ngram_counts, token_counts, n):
    vocabulary_size = len(set(token_counts))
    smoothed_probabilities = {}
    for ngram, count in ngram_counts.items():
        prefix = ngram[:-1]
        prefix_count = token_counts[prefix]
        smoothed_count = count + 1
        smoothed_probability = smoothed_count / (prefix_count + vocabulary_size)
        smoothed_probabilities[ngram] = smoothed_probability
    return smoothed_probabilities

def main(file_path, n):
    tokens = load_and_preprocess_text(file_path)
    ngram_counts, token_counts = count_ngrams_and_tokens(tokens, n)
    smoothed_probabilities = calculate_smoothed_probabilities(ngram_counts, token_counts, n)
    print(list(smoothed_probabilities.items())[:10])

file_path = '/content/Tarzan.txt'
n_values = [3, 5]

for n in n_values:
    print(f"Using {n}-gram:")
    main(file_path, n)


Using 3-gram:
[(('the', 'project', 'gutenberg'), 0.004967855055523086), (('project', 'gutenberg', 'ebook'), 0.0005844535359438924), (('gutenberg', 'ebook', 'of'), 0.0002922267679719462), (('ebook', 'of', 'tarzan'), 0.0002922267679719462), (('of', 'tarzan', 'lord'), 0.0002922267679719462), (('tarzan', 'lord', 'of'), 0.0016072472238457044), (('lord', 'of', 'the'), 0.0018994739918176504), (('of', 'the', 'jungle'), 0.004967855055523086), (('the', 'jungle', 'this'), 0.0002922267679719462), (('jungle', 'this', 'ebook'), 0.0002922267679719462)]
Using 5-gram:
[(('the', 'project', 'gutenberg', 'ebook', 'of'), 0.0002922267679719462), (('project', 'gutenberg', 'ebook', 'of', 'tarzan'), 0.0002922267679719462), (('gutenberg', 'ebook', 'of', 'tarzan', 'lord'), 0.0002922267679719462), (('ebook', 'of', 'tarzan', 'lord', 'of'), 0.0002922267679719462), (('of', 'tarzan', 'lord', 'of', 'the'), 0.0002922267679719462), (('tarzan', 'lord', 'of', 'the', 'jungle'), 0.0016072472238457044), (('lord', 'of', 'the'